
<a href="https://www.kaggle.com/kernels/fork/18483792" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# Run heptabot `xxl` model on Kaggle TPU

This notebook lets you to process data with our `xxl` model in Kaggle TPU environment. Kaggle is the only widely available environment that provides access `v3-8` TPU topology required by T5 11B checkpoint. With Kaggle, you can efficiently run our `xxl` model.

As Colab has recently switched to Python 3.7 and our dependency `spaCy 1.9.0` supports only Python 3.6, we use `mamba` to ensure that we get the right packages in our environment. To get `mamba`, you should execute the following cell (click the '▷' button). Please note that the runtime will restart after that, so don't schedule the rest of the cells to execute just yet.

After your runtime is restarted, execute the following cell to set some environmental variables:



In [ ]:
import os

model_type = "xxl"
# The steps are largely the same between medium and xxl models. 
# However, we keep this, as it is advantageous to run medium model in Google Colab, while xxl – in Kaggle, and these environments have their differences

os.environ["MODEL_PLACE"] = "tpu"
os.environ["HPT_MODEL_TYPE"] = model_type

if model_type == "xxl":
    os.environ["CHECKPOINT_STEP"] = "1014000"
    os.environ["TPU_TOPOLOGY"] = "v3-8"
else:
    os.environ["CHECKPOINT_STEP"] = "1003800"
    os.environ["TPU_TOPOLOGY"] = "v2-8"

## Prepare environment

Now click the '▷' on this group of cells. The code below will install the environment for `heptabot` and needs around 10 minutes to execute.

In [ ]:
!pip install -qq t5==0.9.0 seqio rouge_score sacrebleu sentencepiece

In [ ]:
!git clone -q https://github.com/lcl-hse/heptabot -b tensorflow
%cd heptabot
!mv scripts/colab-run/* .
!mv scripts/tpu-run/* .
!chmod +x colab_run.sh
!chmod +x tpu_run.sh
!sed -i -e 's/mamba/conda/g' colab_setup.sh
!chmod +x colab_setup.sh

In [ ]:
!time bash colab_setup.sh

In [ ]:
!mkdir output
!cp -r static output
!mkdir raw

In [ ]:
import subprocess
from time import sleep

subprocess.Popen(["/bin/bash", os.path.join(os.path.realpath("."), "colab_run.sh")])
sleep(70)

In [ ]:
import re
import os
import pickle
import IPython

## Get the texts

The textual data is downloaded in this part. Here we use 3 essays from [REALEC](https://realec.org/) as example data; you should, however, change this part to process the texts you need.

In [ ]:
!mkdir input

!wget -q "https://realec.org/ajax.cgi?action=downloadFile&collection=%2Fexam%2FExam2015%2F&document=2015_KT_12_2&extension=txt&protocol=1" -O ./input/KT_12_2.txt
!wget -q "https://realec.org/ajax.cgi?action=downloadFile&collection=%2Fexam%2FExam2014%2F&document=2014_ESha_2_1&extension=txt&protocol=1" -O ./input/ESha_2_1.txt
!wget -q "https://realec.org/ajax.cgi?action=downloadFile&collection=%2Fexam%2FExam2016%2F&document=2016_LKa_2_2&extension=txt&protocol=1" -O ./input/LKa_2_2.txt

files = ["KT_12_2.txt", "ESha_2_1.txt", "LKa_2_2.txt"]
textdict = {}

for f in files:
  with open(os.path.join("input", f), "r", encoding="utf-8") as infile:
    textdict[f[:-4]] = infile.read()

**Important**: If you got here from the error page on `heptabot` website stating "*In order to maintain server resources and stable uptime, we limit the amounts of data that can be processed via our Web interface*", uncomment the following code (remove all the number signs) and upload the `generated.txt` file you got from our website:

In [ ]:
#from google.colab import files
#files.upload()

#textdict = {}

#with open("generated.txt", "r", encoding="utf-8") as infile:
  #textdict["generated"] = infile.read()

In other cases, we recommend to put your files into the **`input`** folder for comprehensibility.

Put all your texts in a `dict` with the name `textdict`, where keys are `str`'s with texts IDs (preferrably filenames without extension), while the actual data is stored also as `txt`'s in values, as such:

In [ ]:
texts = textdict

assert all(type(k) is str for k in texts.keys())
assert all(type(v) is str for v in texts.values())

## Process data with `heptabot`

The actual `heptabot` magic is performed here!

**Important**: please choose the appropriate task type in the following cell. While `correction`, the default, is used to correct whole essays and only its pipeline incororates the error classification subroutine, you may also want to perform sentencewise correction. In this case, choose one of the identifiers of the relevant GEC tasks: `jfleg` (trained on JFLEG data) is for general sentencewise correction and should provide more diverse results, while `conll` (trained on CONLL-14 competition) and `bea` (trained on BEA-2019 competition) correct mainly grammar-related errors, for which case the grammar parsing data is appended to the sentence in the corresponding pipeline. Please note that `heptabot` expects whole paragraphs of text as data for `correction` and sentence-by-sentence structured data for other tasks, so make sure your file(s) contain single sentences separated by newlines if you wish to perform any other task than `correction`.

In [ ]:
task_type = "correction"  #@param ["correction", "jfleg", "conll", "bea"] 

In [ ]:
import random
chosen_one = random.choice(list(texts.keys()))

print(texts[chosen_one])

In [ ]:
pickledata = (task_type, texts)

with open("./raw/process_texts.pkl", "wb") as outpickle:
  pickle.dump(pickledata, outpickle)

In order to get the advantage of using TPU, we split our process in three parts. First, we prepare our texts by splitting them into batches required by `heptabot`:

In [ ]:
%%bash
source activate heptabot
python batchify_input.py

Then we call the TPU process (this is where our `xxl` model runs inference on the texts):

In [ ]:
%%bash
python tpu_model_run.py

And then, finally, we glue our processed texts back together to produce the outputs:

In [ ]:
%%bash
source activate heptabot
python process_output.py

## Display the results

Finally, in this section you can display the processed results.

In [ ]:
#This cell contains a function that makes pretty displaying work
def prepare_display(filekey):
  template = """<html><head>
	<meta charset="utf-8">
	<meta content="IE=edge" http-equiv="X-UA-Compatible">
	<meta content="width=device-width, initial-scale=1" name="viewport">
	<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
	<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
	<script src="https://ajax.googleapis.com/ajax/libs/jquery/2.1.3/jquery.min.js"></script>
	<link href="https://getbootstrap.com/docs/3.3/dist/css/bootstrap.min.css" rel="stylesheet"><!-- IE10 viewport hack for Surface/desktop Windows 8 bug -->
	<link href="https://getbootstrap.com/docs/3.3/assets/css/ie10-viewport-bug-workaround.css" rel="stylesheet"><!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
	<link href="https://fonts.googleapis.com/css2?family=Kanit&family=Mukta&family=PT+Sans&family=PT+Serif&family=Ubuntu+Mono&display=swap" rel="stylesheet">
<style>
{0}
</style>
<script type="text/javascript">
{1}
</script>
</head>
<body>
<div class="header2">{2}</div><br>
{3}
</body></html>"""

  with open("static/result/style.css", "r") as inhtml:
    style = inhtml.read()
  with open("static/result/engine.js", "r") as inhtml:
    script = inhtml.read().replace("var em;", "var em=18;").replace("elemtitle.style.left = varleft + 'px';", "elemtitle.style.left = varleft - 70 + 'px';")
  with open(os.path.join('./output', filekey + ".html"), "r") as inhtml:
    htmlcont = inhtml.read()
  tt = re.search(r'<div class="header2">(.*?)</div>', htmlcont, flags=re.DOTALL).group(1)
  result_div = re.search(r'<div id="resulta".*?\n', htmlcont).group(0)
  outcont = template.format(style, script, tt, result_div)
  with open("display.html", "w", encoding="utf-8") as outhtml:
    outhtml.write(outcont)

In [ ]:
#@markdown Enter the desired text ID below to pretty-print the result
display_id = chosen_one  #@param {type: "string"}

prepare_display(display_id)
IPython.display.HTML(filename='display.html')

## Download the results

Now, you may also want to get the texts processed by `heptabot`. The code below will generate a link to download the processed texts directly to your computer: unzip them to view the results as they would be displayed in the web version.

In [ ]:
!zip -q heptabot_processed.zip -r output

In [ ]:
#from IPython.display import FileLink

#FileLink(r'heptabot_processed.zip')